# 모형결합 (model combining)
## 앙상블 방법론 (ensemble methods)
* 예측 성능을 향상시키기 위해서 하나의 모형이 아닌 여러개의 모형을 결합하는 방법
* 장점 : 과적합 방지 등을 통해 성능 향상
* 단점 : 단일 모형을 사용하는 것 보다 계산량 증가

# 앙상블 방법론 - 취합(aggregation) & 부스팅(boosting)
## 취합(aggregation)
* 사용할 모형의 집합을 처음부터 **고정**함
* Ex : 다수결(Majority Voting), 배깅(Bagging), 랜덤포레스트(Random Forest)
    -  다수결 방법(Majority Voting)
        * 여러 종류의 모형들 중 가장 좋은 결과를 가진 모델 채택
        * 채택 방법 :
            * Hard Voting
                * 단순 투표, 가장 많이 나온 결과를 채택 (Dafault)
            * Soft Voting
                * 가중치 투표, 개별 모형의 조건부 확률들을 합한 것들 중 가장 큰 것을 재택함
    - 배깅(Bagging)
        * 같은 모형을 사용하지만 같은 데이터 샘플을 중복으로 사용하여, 서로 다른 결과를 출력하는 다수의 모형을 사용하는 방법
        * BaggingClassifier
            - base_estimator : 기본 모형
            - n_estimators : 모형 개수(default = 10)
            - bootstrap : 데이터 중복 사용 여부(default = True)
            - max_samples : 데이터 샘플 중 선택할 샘플의 수 또는 비율(default = 1.0)
            - bootstrap_features : 특징 차원의 중복 사용 여부(default = False)
            - max_features : 다차원 독립 변수 중 선택할 차원의 수 혹은 비율 (default = 1.0)

## 부스팅(Boosting)
* 사용할 모형을 **점진적으로 늘려가는** 방법
* Ex : 에이다부스트(AdaBoost), 그레디언트 부스트(GradientBoost)

### [실습] 다수결 방법 - 신용카드 데이터셋

In [11]:
import pandas as pd
df = pd.read_csv('datasets/creditcard.csv')

# 독립변수 컬럼 리스트 : 처음컬럼빼고 다 가져오기
train_cols = df.columns[1:-1]

# 독립변수, 종속변수 설정
X = df[train_cols]
y = df["Class"]

#언더 샘플링
from imblearn.under_sampling import RandomUnderSampler
X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)
X_samp = pd.DataFrame(data=X_sample, columns=train_cols)
y_samp = pd.DataFrame(data=y_sample, columns=['Class'])
df2 = pd.concat([X_samp, y_samp], axis=1)

#독립변수, 종속변수
X = X_samp[train_cols]
y = y_samp["Class"]

#학습&검증용 데이터
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#여러 모델 선언
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

model1 = LogisticRegression(random_state= 1, max_iter=1000)
model2 = DecisionTreeClassifier(random_state= 1)
model3 = KNeighborsClassifier(n_neighbors= 2)

# estimators : 개별 모형 목록, 리스트나 named parameter 형식으로 입력
# voting : 문자열 {hard, soft} hard voting 과 soft voting 선택. 디폴트 hard
# 3개 모델 합친 모형
ensemble = VotingClassifier(estimators=[('lr', model1), ('tree', model2), ('knn', model3)], voting='soft')

for model in (model1, model2, model3, ensemble):
    print(model)
    model.fit(X_train, y_train)
    print("학습용:", model.score(X_train, y_train))
    print("검증용:", model.score(X_test, y_test))
    print()



LogisticRegression(max_iter=1000, random_state=1)
학습용: 0.9529860228716646
검증용: 0.9289340101522843

DecisionTreeClassifier(random_state=1)
학습용: 1.0
검증용: 0.9289340101522843

KNeighborsClassifier(n_neighbors=2)
학습용: 0.9428208386277002
검증용: 0.9289340101522843

VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=1)),
                             ('tree', DecisionTreeClassifier(random_state=1)),
                             ('knn', KNeighborsClassifier(n_neighbors=2))],
                 voting='soft')
학습용: 1.0
검증용: 0.9441624365482234



### [실습] 모형결합 (배깅)

In [8]:
import pandas as pd

df = pd.read_csv("datasets/customer.csv")


df["Churn"].value_counts()

train_cols = df.columns[0:16]

X = df[train_cols]
y = df["Churn"]

from imblearn.under_sampling import RandomUnderSampler
X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)

X_samp = pd.DataFrame(data=X_sample, columns=train_cols)
y_samp = pd.DataFrame(data=y_sample, columns=['Churn'])
df_samp = pd.concat([X_samp, y_samp], axis=1)

df_samp["Churn"].value_counts()

X = df_samp[train_cols]
y = df_samp["Churn"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
model1 = DecisionTreeClassifier(random_state=0)
model2 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, random_state=0)

# for model in (model1, model2):
#     print(model)
#     model.fit(X_train, y_train)
#     print("학습용:", model.score(X_train, y_train))
#     print("검증용:", model.score(X_test, y_test))
#     print()

### [실습] 랜덤포레스트
#### 랜덤포레스트
- 의사결정나무를 개별 모형으로 사용하는 모형 결합 방법
- 배깅의 일종으로, 배깅과 다르게 설명변수를 무작위로 선택함으로써 트리의 다양성을 확보해 모형간의 상관관계를 줄이고자 함
- 배깅은 모형 종류에 제한이 없으나, **랜덤포레스트는 의사결정나무 모형만을 사용함**
- 독립변수의 차원을 랜덤하게 감소시킨 후 독립변수를 선택하는 방법

In [7]:
import pandas as pd
df = pd.read_csv('datasets/heart.csv')

df["target"].value_counts()

df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')